In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
# You may change the mhealth_activity module but your algorithm must support the original version
from mhealth_activity import Recording, Trace, Activity, WatchLocation, Path
from tqdm import tqdm
from multiprocessing import Pool
import pickle
# For interactive plots, uncomment the following line
# %matplotlib widget

In [4]:
create_data_pickle = False
if create_data_pickle:
    files = os.listdir('data/train')
    list_of_dicts = []
    types_to_include = ['ax', 'ay', 'az', 'phone_ax', 'phone_ay', 'phone_az', 'speed', 'longitude', 'latitude', 'altitude', 'phone_steps']

    for file in tqdm(files):
        Dict = {}
        d = Recording(os.path.join('data/train',file))

        Dict['labels'] = d.labels
        for data_type in types_to_include:
            if data_type in d.data.keys():
                Dict[data_type] = d.data[data_type]
        list_of_dicts.append(Dict)

    data = pd.DataFrame(list_of_dicts)
    data.to_pickle(path='data/pickled_and_sorted_training_data.pkl.zst', compression={'method': 'zstd'})
else:
    data = pd.read_pickle('data/pickled_and_sorted_training_data.pkl.zst')

In [3]:
# for i in range(20):
    # print(data.l['labels'])

# d = Recording('data/train/train_trace_000.pkl')
# d.__repr__()
print(data.loc[0]['ax'].values)


[-0.87182617 -0.87255859 -0.87597656 ... -0.41650391 -0.40869141
 -0.41235352]


In [ ]:
def norm(input):
    return (np.sqrt(input[1]['ax'].values**2 + input[1]['ay'].values**2 + input[1]['az'].values**2))

def calc_accel_mag(input):

    mag = []
    with Pool(8) as p:
        mag = p.map(norm, data.iterrows())

    # for trace in data.iterrows():
        # print((trace[1]['ax'].values))
        
    print(mag)
    return mag
# print(len(data.loc))

calc_a_mag = True
if calc_a_mag:

    accel_mag = calc_accel_mag(data)
    with open('data/accel_mag_train.pkl', 'wb') as f:
        pickle.dump(accel_mag, f)
        f.close()

else:
    file = open('data/accel_mag_train.pkl', 'rb')
    accel_mag =  pickle.load(file)
    file.close()
    print("aaa")
    print(accel_mag)
    

    

In [2]:
#rinse and repeat for test data
create_data_pickle = True
if create_data_pickle:
    files = os.listdir('data/test')
    list_of_dicts = []
    types_to_include = ['ax', 'ay', 'az', 'phone_ax', 'phone_ay', 'phone_az', 'speed', 'longitude', 'latitude', 'altitude', 'phone_steps']

    for file in tqdm(files):
        Dict = {}
        d = Recording(os.path.join('data/test',file))

        Dict['labels'] = d.labels
        for data_type in types_to_include:
            if data_type in d.data.keys():
                Dict[data_type] = d.data[data_type]
        list_of_dicts.append(Dict)

    data = pd.DataFrame(list_of_dicts)
    data.to_pickle(path='data/pickled_and_sorted_test_data.pkl.zst', compression={'method': 'zstd'})
else:
    data = pd.read_pickle('data/pickled_and_sorted_test_data.pkl.zst')

100%|██████████| 280/280 [00:24<00:00, 11.50it/s]
